In [1]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

In [5]:
# Data
suppliers = ['A', 'B', 'C', 'D']
supply_capacity = [500, 600, 400, 300]
cost_per_unit = [10, 12, 8, 11]
carbon_footprint_per_unit = [2, 1.5, 2.5, 1.8]
total_demand = 1000

# Goals
cost_goal = 10000
carbon_goal = 200

# Weights for deviation variables
weight_cost = 0.6
weight_carbon = 0.4

# Create a new model
model = gp.Model("weighted_goal_programming")

# Create variables
x = model.addVars(suppliers, vtype=GRB.INTEGER, name="x", lb=0)
d1_plus = model.addVar(name="d1_plus", lb=0)
d1_minus = model.addVar(name="d1_minus", lb=0)
d2_plus = model.addVar(name="d2_plus", lb=0)
d2_minus = model.addVar(name="d2_minus", lb=0)

# Set objective
model.setObjective(weight_cost * (d1_plus + d1_minus) + weight_carbon * (d2_plus + d2_minus), GRB.MINIMIZE)

# Add constraints
model.addConstr(gp.quicksum(x[suppliers[i]] for i in range(len(suppliers))) == total_demand, "total_demand")
for i in range(len(suppliers)):
    model.addConstr(x[suppliers[i]] <= supply_capacity[i], f"supply_capacity_{suppliers[i]}")

# Goals with deviation variables
model.addConstr(gp.quicksum(cost_per_unit[i] * x[suppliers[i]] for i in range(len(suppliers))) + d1_minus - d1_plus == cost_goal, "cost_goal")
model.addConstr(gp.quicksum(carbon_footprint_per_unit[i] * x[suppliers[i]] for i in range(len(suppliers))) + d2_minus - d2_plus == carbon_goal, "carbon_goal")

# Optimize model
model.optimize()

# Display results
if model.status == GRB.OPTIMAL:
    print("Optimal solution found:")
    for i in range(len(suppliers)):
        print(f"Purchase from supplier {suppliers[i]}: {x[suppliers[i]].X} units")
    print(f"Total cost: {gp.quicksum(cost_per_unit[i] * x[suppliers[i]].X for i in range(len(suppliers))).getValue()}")
    print(f"Total carbon footprint: {gp.quicksum(carbon_footprint_per_unit[i] * x[suppliers[i]].X for i in range(len(suppliers))).getValue()}")
    print(f"Cost deviation: +{d1_plus.X}, -{d1_minus.X}")
    print(f"Carbon footprint deviation: +{d2_plus.X}, -{d2_minus.X}")
else:
    print("No optimal solution found.")

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i7-8665U CPU @ 1.90GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 7 rows, 8 columns and 20 nonzeros
Model fingerprint: 0x5011398f
Variable types: 4 continuous, 4 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [4e-01, 6e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+02, 1e+04]
Presolve removed 5 rows and 2 columns
Presolve time: 0.00s
Presolved: 2 rows, 6 columns, 9 nonzeros
Variable types: 2 continuous, 4 integer (0 binary)
Found heuristic solution: objective 776.0000000

Root relaxation: objective 7.200000e+02, 2 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0     720.0000000 